# DNBLab Jupyter Notebook zur SRU Abfrage

## SRU - Schnittstellenabfrage, Datenauslieferung und Ergebnisanzeige

Lassen Sie uns gemeinsam folgenden Markdown *hier* einfügen:

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage über die SRU-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Abfrage und Ausgabe der Daten in MARC21-xml zur weiteren Verarbeitung mit Python. 

## Ganz wichtig, Zwischenspeichern nicht vergessen!

## Arbeitsumgebung einrichten! <a class="anchor" id="Teil1"></a>

In [1]:
import urllib.parse
from pyodide.http import open_url, pyfetch
from js import fetch
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## Abfrage über alle Datensätze... <a class="anchor" id="Teil2"></a>

In [2]:
async def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
    r_text = await r.text()
    xml = soup(r_text, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
            r_text = await r.text()
            xml = soup(r_text, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Felder Titel und Links zu den Objekten durchsuchen... <a class="anchor" id="Teil3"></a>

In [15]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # link
    link = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        link = "unkown"
        
          # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
    
    meta_dict = {"idn":idn,
                 "titel":titel,
                 "link":link}
    
    return meta_dict

# Abfrage nach Suchwort "Biene" im Titel + Links zu frei verfügbaren Objekten!

In [16]:
records = await dnb_sru('tit=Biene and location=onlinefree')
print(len(records), 'Ergebnisse')

17 Ergebnisse


## Anzeige der Treffer in einer Tabelle...  <a class="anchor" id="Teil4"></a>

In [17]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,titel,link
0,1283345676,"Verkauf der ""Biene"" an Franz Wagner in Leipzig...",https://nbn-resolving.org/urn:nbn:de:101:1-202...
1,1299731961,Die Biene und der Blitz - die eigenwillige K...,https://nbn-resolving.org/urn:nbn:de:bvb:384-o...
2,1246522276,Info Bee at UNICARagil: Support from Above for...,https://doi.org/10.18154/RWTH-2021-10614
3,1213898609,Die Biene - Illustrierte Schüler*inarbeit - ...,https://doi.org/10.18442/140
4,1156534089,Aus der Wabe in die Welt: Biene macht Kultur. ...,https://nbn-resolving.org/urn:nbn:de:bvb:20-op...
5,118258280X,"[Rezension zu:] Viel, Bernhard: Der Honigsamml...",https://nbn-resolving.org/urn:nbn:de:hebis:30:...
6,1095550535,[Referentenmaterial] / Gesellschaft zur Verbre...,https://nbn-resolving.org/urn:nbn:de:101:1-201...
7,1263070647,Harald Weiss (Hg.): 100 Jahre Biene Maja: Vom ...,https://doi.org/10.17192/ep2015.2.3594
8,1076865429,Harald Weiss (Hg.): 100 Jahre Biene Maja: Vom ...,https://doi.org/10.17192/ep2015.2.3594
9,1046981080,Biene und Honig im pharaonischen Ägypten,https://nbn-resolving.org/urn:nbn:de:hebis:77-...


# Speichern der Ergebnisse als CSV-Datei!

In [18]:
df.to_csv("SRU_Titel.csv", index=False)